In [182]:
import pandas as pd
import delorean
import re
import argparse
from decimal import Decimal
from parse import parse 

In [183]:

input_text = """AFTER THE CLOSE OF THE SECOND QUARTER, OUR COMPANY, CASTAÑACORP
                HAS ACHIEVED A GROWTH IN THE REVENUE OF 7.47%. THIS IS IN LINE
                WITH THE OBJECTIVES FOR THE YEAR. THE MAIN DRIVER OF THE SALES HAS 
                BEEN
                THE NEW PACKAGE DESIGNED UNDER THE SUPERVISION OF OUR MARKETING 
                DEPARTMENT.
                OUR EXPENSES HAS BEEN CONTAINED, INCREASING ONLY BY 0.7%, THOUGH THE 
                BOARD
                CONSIDERS IT NEEDS TO BE FURTHER REDUCED. THE EVALUATION IS 
                SATISFACTORY
                AND THE FORECAST FOR THE NEXT QUARTER IS OPTIMISTIC. THE BOARD 
                EXPECTS
                AN INCREASE IN PROFIT OF AT LEAST 2 MILLION DOLLARS.
            """


In [184]:
words = input_text.split()

In [185]:
redactaed = [''.join('X' if w.isdigit() else w for w in word) for word in words]

In [186]:
ascii_text = [word.encode('ascii', errors = 'replace').decode('ascii') for word in redactaed]

In [187]:
newlines = [word + '\n' if word.endswith('.') else word for word in ascii_text]

In [188]:
line_size = 80 
lines = []
line = ''
for word in newlines:
    if line.endswith('\n') or len(line) + len(word) + 1 > line_size:
        lines.append(line)
        line = ''
    line = line + ' ' + word

In [189]:
lines = [line.title() for line in lines]

In [190]:
result = '\n'.join(lines)

In [191]:
print(result)

 After The Close Of The Second Quarter, Our Company, Casta?Acorp Has Achieved A
 Growth In The Revenue Of X.Xx%.

 This Is In Line With The Objectives For The Year.

 The Main Driver Of The Sales Has Been The New Package Designed Under The
 Supervision Of Our Marketing Department.

 Our Expenses Has Been Contained, Increasing Only By X.X%, Though The Board
 Considers It Needs To Be Further Reduced.

 The Evaluation Is Satisfactory And The Forecast For The Next Quarter Is
 Optimistic.



In [192]:
log = '[2018-05-05T10:58:41.504054] - SALE - PRODUCT: 1345 - PRICE: $09.99'

In [193]:
class PriceLog(object):
    def __init__(self, timestamp, product_id, price):
        self.timestamp = timestamp
        self.product_id = product_id
        self.price = price
    def __repr__(self):
        return '<PriceLog ({}, {}, {})>'.format(self.timestamp,
                                                self.product_id,
                                                self.price)
    @classmethod
    def parse(cls, text_log):
        '''
        Parse from a text log with the format
        [<Timestamp>] - SALE - PRODUCT: <product id> - PRICE: $<price>
        to a PriceLog object
        '''
        divide_it = text_log.split(' - ')
        tmp_string, _, product_string, price_string = divide_it
        timestamp = delorean.parse(tmp_string.strip('[]'))
        product_id = int(product_string.split(':')[-1])
        price = Decimal(price_string.split('$')[-1])
        return cls(timestamp=timestamp, product_id=product_id, price=price)

In [194]:
PriceLog.parse(log)

<PriceLog (Delorean(datetime=datetime.datetime(2018, 5, 5, 10, 58, 41, 504054), timezone='UTC'), 1345, 9.99)>

## Regex

In [195]:
re.search(r'some', 'some log')

<re.Match object; span=(0, 4), match='some'>

In [196]:
re.search(r'^log', 'logs some')

<re.Match object; span=(0, 3), match='log'>

In [197]:
re.search(r'log$', 'some log')

<re.Match object; span=(5, 8), match='log'>

In [198]:
strings = 'something in the things she shows me'

In [199]:
match = re.search(r'things', strings)

In [200]:
strings[:match.start()], strings[match.start():match.end()], strings[match.end():]

('something in the ', 'things', ' she shows me')

In [201]:
matche = re.search(r'\bthing', strings)
matche

<re.Match object; span=(17, 22), match='thing'>

In [202]:
strings[:matche.start()], strings[matche.start():matche.end()], strings[matche.end():]

('something in the ', 'thing', 's she shows me')

In [203]:
re.search(r'[\d-]+', 'the phone number is 43-567-890').group()

'43-567-890'

In [204]:
re.search(r'\S+@\S+', 'my email is email.134@google.com').group()

'email.134@google.com'

In [205]:
re.search(r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",'john@smith@test.com')

<re.Match object; span=(5, 19), match='smith@test.com'>

## Going deeper into regex

In [206]:
match = re.search(r'the phone number is ([\d-]+)', '23: the phone number is 1234-567-890').group(1)
match

'1234-567-890'

In [207]:
pattern = re.compile(r'The answer to question (\w+) is (yes|no)', re.IGNORECASE)

In [208]:
pattern.search('Naturally, the answer to question 3b is no').groups()


('3b', 'no')

In [209]:
pattern = re.compile(r'([A-Z][\w\s]+?).(TX|OR|OH|MI)')

In [210]:
text ='the jackalopes are the team of Odessa,TX while the knights are native of Corvallis OR and the mud hens come from Toledo.OH; the whitecaps have their base in Grand Rapids,MI'

In [211]:
list(pattern.finditer(text))

[<re.Match object; span=(31, 40), match='Odessa,TX'>,
 <re.Match object; span=(73, 85), match='Corvallis OR'>,
 <re.Match object; span=(113, 122), match='Toledo.OH'>,
 <re.Match object; span=(157, 172), match='Grand Rapids,MI'>]

In [212]:
re.search(r'([A-Z][\w\s]+?).(TX|OR|OH|MI)', 'This is a test, Escanaba MI')

<re.Match object; span=(16, 27), match='Escanaba MI'>

In [213]:
pattern = re.compile(r'(?P<city>[A-Z][\w\s]+?).(?P<state>TX|OR|OH|MN)')

In [214]:
match = pattern.search(text)

In [215]:
match.groupdict()

{'city': 'Odessa', 'state': 'TX'}

In [216]:
match.group('state')

'TX'

In [217]:
match.group(1), match.group(2)

('Odessa', 'TX')

In [218]:
def main(number):
    print('#' * number)

In [219]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('number', type = int, help = 'A number')
    args = parser.parse_args()
    main(args.number)

usage: ipykernel_launcher.py [-h] number
ipykernel_launcher.py: error: argument number: invalid int value: 'C:\\Users\\USER\\AppData\\Roaming\\jupyter\\runtime\\kernel-f8a93178-9eed-48a7-ac1d-699be25f542c.json'


AssertionError: 

In [1]:
import smtplib
from email.mime.text import MIMEText

In [4]:
def send_email(subject, body, sender, recipients, password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp_server.login(sender, password)
    smtp_server.sendmail(sender, recipients, msg.as_string())
    smtp_server.quit()
subject = "Email Subject"
body = "This is the body of the text message"
sender = "sperolking@gmail.com"
recipients = ['harrisononh3@gmail.com']
password = "kingsleYY1@"
send_email(subject, body, sender, recipients, password)

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials g16-20020a5d4890000000b002c56a991971sm6697293wrq.76 - gsmtp')